In [ ]:
import nltk
nltk.download('stopwords') # Для удаления брани

import numpy as np 
import pandas as pd

import sklearn 
from sklearn.pipeline import FeatureUnion # Для объединения признаков
from sklearn.model_selection import train_test_split

# Для дополнительных преобразований числовых и текстовых признаков
from sklearn.base import BaseEstimator, TransformerMixin 

# Ансамблевый классификатор "Случайный лес"
from sklearn.ensemble import RandomForestClassifier

# Непосредственно пайплайн для подготовки датафрейма
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer

# Стандартизация дополнительного признака
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Библиотеки быстро эволюционируют, потому указывается рабочая версия, с которой работал автор на момент написания статьи. Для быстрого дебаггинга используйте команду для установки явной версии '!pip install [библиотека]==[версия]'.


In [ ]:
print(nltk.__version__)
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)

3.2.5
1.19.5
1.1.5
0.22.2.post1


In [ ]:
df = pd.read_csv('https://www.dropbox.com/s/pvqawholxi3l2uq/pipeline-author-identification.csv?dl=1')

# Опустим пустые ячейки
df.dropna(axis = 0)

# Зададим индекс
df.set_index('id', inplace = True)
df.head()

,text,author,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,,,
id26305,"This process, however, afforded me no means of...",EAP,this process however afforded me no means of a...,224,41,21,6.380952,4
id17569,It never once occurred to me that the fumbling...,HPL,it never once occurred to me that the fumbling...,70,14,6,6.166667,0
id11008,"In his left hand was a gold snuff box, from wh...",EAP,in his left hand was a gold snuff box from whi...,195,36,19,5.947368,4
id27763,How lovely is spring As we looked from Windsor...,MWS,how lovely is spring as we looked from windsor...,202,34,21,6.476190,3
id12958,"Finding nothing else, not even gold, the Super...",HPL,finding nothing else not even gold the superin...,170,27,16,7.187500,2


In [ ]:
# Разделим признаки на числовые (numeric_features) и остальные (features)
features = [c for c in df.columns.values if c  not in ['id', 'text', 'author']]
numeric_features = [c for c in df.columns.values if c  not in ['id', 'text', 'author', 'processed']]

# Выберем целевую переменную – мы хотим идентифицировать автора поста по его стилю
target = 'author'

# Разделим датасет на тренировочную и тестовую части случайным образом в пропорции 67:33
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size = 0.33, random_state = 42)
X_train.head()

,processed,length,words,words_not_stopword,avg_word_length,commas
id,,,,,,
id19417,this panorama is indeed glorious and i should ...,91,18,6,6.666667,1
id09522,there was a simple natural earnestness about h...,240,44,18,6.277778,4
id22732,who are you pray that i duc de lomelette princ...,387,74,38,5.552632,9
id10351,he had gone in the carriage to the nearest tow...,118,24,11,5.363636,0
id24580,there is no method in their proceedings beyond...,71,13,5,7.000000,1


In [ ]:
class TextSelector(BaseEstimator, TransformerMixin):
    # Функция, выбирающая каждый текстовый признак датасета 
    # для дополнительных преобразований
    
    def __init__(self, key):
        self.key = key

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    # Функция, выбирающая каждый числовой признак датасета 
    # для дополнительных преобразований
       
    def __init__(self, key):
        self.key = key

    def fit(self, X, y = None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [ ]:
text = Pipeline([
                ('selector', TextSelector(key = 'processed')),
                ('tfidf', TfidfVectorizer(stop_words = 'english'))
            ])

# Передадим модели часть данных
text.fit_transform(X_train)

<13117x21516 sparse matrix of type '<class 'numpy.float64'>'
	with 148061 stored elements in Compressed Sparse Row format>

In [ ]:
length =  Pipeline([
                ('selector', NumberSelector(key = 'length')),
                ('standard', StandardScaler())
            ])

# Передадим модели еще часть
length.fit_transform(X_train)

array([[-0.50769254],
       [ 0.88000324],
       [ 2.24907223],
       ...,
       [-0.46112557],
       [-0.14447015],
       [-0.39593181]])

In [ ]:
feats = FeatureUnion([('text', text), # Текст поста              
                      ('length', length), # Длина поста
                      ('words', words), # Список уникальных использованных слов
                      ('words_not_stopword', words_not_stopword), # Очищенный список без ругательств
                      ('avg_word_length', avg_word_length), # Средняя длина слова
                      ('commas', commas)]) # Количество запятых


# Объединим результаты нескольких преобразованных переменных в единый набор данных. 
# Мы сделаем конвейер для каждой переменной, затем объединим их.
words =  Pipeline([
                ('selector', NumberSelector(key = 'words')),
                ('standard', StandardScaler())
            ])
words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key = 'words_not_stopword')),
                ('standard', StandardScaler())
            ])
avg_word_length =  Pipeline([
                ('selector', NumberSelector(key = 'avg_word_length')),
                ('standard', StandardScaler())
            ])
commas =  Pipeline([
                ('selector', NumberSelector(key = 'commas')),
                ('standard', StandardScaler()),
            ])

In [ ]:
feature_processing = Pipeline([('feats', feats)])

# Передадим модели еще часть данных
feature_processing.fit_transform(X_train)

<13117x21521 sparse matrix of type '<class 'numpy.float64'>'
	with 213646 stored elements in Compressed Sparse Row format>

In [ ]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

# Теперь модель получила все учебные данные
pipeline.fit(X_train, y_train)

# Определим точность
preds = pipeline.predict(X_test)
np.mean(preds == y_test)

0.6792014856081708